Find solutions $(a,b,c,d)$ for the diophantine equation $a(b+1)c(d+1)=(a+1)b(c+1)d$, see [this post](https://math.stackexchange.com/questions/4389370/positive-integer-solutions-to-product/4397280) at MSE.

In [2]:
a_max=10
for a in range(1,a_max+1):
    for b in range(1,a):
        for c in range(1,(-b-a*b)//(-a+b)):
            num = -a*c-a*b*c
            denom = -b-a*b+a*c-b*c
            d = num // denom
            if num == d * denom:
                print([a,b,c,d])


[2, 1, 1, 2]
[2, 1, 2, 8]
[3, 1, 1, 3]
[3, 2, 2, 3]
[3, 2, 4, 9]
[3, 2, 5, 15]
[3, 2, 6, 27]
[3, 2, 7, 63]
[4, 2, 2, 4]
[4, 2, 3, 9]
[4, 2, 4, 24]
[4, 3, 3, 4]
[4, 3, 5, 8]
[4, 3, 7, 14]
[4, 3, 9, 24]
[4, 3, 10, 32]
[4, 3, 11, 44]
[4, 3, 12, 64]
[4, 3, 13, 104]
[4, 3, 14, 224]
[5, 2, 2, 5]
[5, 2, 3, 15]
[5, 3, 3, 5]
[5, 3, 4, 8]
[5, 3, 6, 20]
[5, 3, 7, 35]
[5, 3, 8, 80]
[5, 4, 4, 5]
[5, 4, 9, 15]
[5, 4, 12, 25]
[5, 4, 14, 35]
[5, 4, 16, 50]
[5, 4, 18, 75]
[5, 4, 19, 95]
[5, 4, 20, 125]
[5, 4, 21, 175]
[5, 4, 22, 275]
[5, 4, 23, 575]
[6, 2, 2, 6]
[6, 3, 3, 6]
[6, 3, 5, 20]
[6, 3, 6, 48]
[6, 4, 4, 6]
[6, 4, 7, 15]
[6, 4, 8, 20]
[6, 4, 9, 27]
[6, 4, 11, 55]
[6, 4, 12, 90]
[6, 4, 13, 195]
[6, 5, 5, 6]
[6, 5, 7, 9]
[6, 5, 14, 24]
[6, 5, 15, 27]
[6, 5, 17, 34]
[6, 5, 20, 48]
[6, 5, 21, 54]
[6, 5, 23, 69]
[6, 5, 25, 90]
[6, 5, 26, 104]
[6, 5, 28, 144]
[6, 5, 29, 174]
[6, 5, 30, 216]
[6, 5, 31, 279]
[6, 5, 32, 384]
[6, 5, 33, 594]
[6, 5, 34, 1224]
[7, 2, 2, 7]
[7, 3, 3, 7]
[7, 3, 4, 14]
[7, 3,

In [1]:
b_max=50
c_max=50

for b in range(1,b_max+1):
    for a in range(1,b+1):
        for c in range(1,c_max):
            num = -a*c-a*b*c
            denom = -b-a*b+a*c-b*c
            d = num // denom
            if num == d * denom:
                print([a,b,c,d])

[1, 1, 1, 1]
[1, 1, 2, 2]
[1, 1, 3, 3]
[1, 1, 4, 4]
[1, 1, 5, 5]
[1, 1, 6, 6]
[1, 1, 7, 7]
[1, 1, 8, 8]
[1, 1, 9, 9]
[1, 1, 10, 10]
[1, 1, 11, 11]
[1, 1, 12, 12]
[1, 1, 13, 13]
[1, 1, 14, 14]
[1, 1, 15, 15]
[1, 1, 16, 16]
[1, 1, 17, 17]
[1, 1, 18, 18]
[1, 1, 19, 19]
[1, 1, 20, 20]
[1, 1, 21, 21]
[1, 1, 22, 22]
[1, 1, 23, 23]
[1, 1, 24, 24]
[1, 1, 25, 25]
[1, 1, 26, 26]
[1, 1, 27, 27]
[1, 1, 28, 28]
[1, 1, 29, 29]
[1, 1, 30, 30]
[1, 1, 31, 31]
[1, 1, 32, 32]
[1, 1, 33, 33]
[1, 1, 34, 34]
[1, 1, 35, 35]
[1, 1, 36, 36]
[1, 1, 37, 37]
[1, 1, 38, 38]
[1, 1, 39, 39]
[1, 1, 40, 40]
[1, 1, 41, 41]
[1, 1, 42, 42]
[1, 1, 43, 43]
[1, 1, 44, 44]
[1, 1, 45, 45]
[1, 1, 46, 46]
[1, 1, 47, 47]
[1, 1, 48, 48]
[1, 1, 49, 49]
[1, 2, 2, 1]
[1, 2, 8, 2]
[2, 2, 1, 1]
[2, 2, 2, 2]
[2, 2, 3, 3]
[2, 2, 4, 4]
[2, 2, 5, 5]
[2, 2, 6, 6]
[2, 2, 7, 7]
[2, 2, 8, 8]
[2, 2, 9, 9]
[2, 2, 10, 10]
[2, 2, 11, 11]
[2, 2, 12, 12]
[2, 2, 13, 13]
[2, 2, 14, 14]
[2, 2, 15, 15]
[2, 2, 16, 16]
[2, 2, 17, 17]
[2, 2, 18, 18]
[2, 2